In [1]:
from keras.utils import to_categorical
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters
import networkx as nx

In [2]:
from sklearn.datasets import load_svmlight_file


def get_data(data):
    data = load_svmlight_file(data)
    return data[0].toarray(), data[1]
data = "./abalone_scale.txt"
X, y = get_data(data)
#y[y == -1] = 0
# Convert labels to one-hot encoding
#y = to_categorical(y)

#data = "./a1a_t"
#X_test,y_test = get_data(data)
#y_test[y_test == -1] = 0
#y_test = to_categorical(y_test)


In [3]:
# Dataset partitioning
def random_split(X, y, n, seed):
    """Equally split data between n agents"""
    rng = np.random.default_rng(seed)
    perm = rng.permutation(y.shape[0])
    X_split = np.array_split(X[perm], n)  #np.stack to keep as a np array
    y_split = np.array_split(y[perm], n)
    return X_split, y_split

In [4]:
no_users = 30

In [5]:
X, y = random_split(X, y, 3, 42)

In [6]:
X1 = X[0][:, 0:4]
X2 = X[1][:, 2: 6]
X3 = X[2][:, 3:]
y1 = y[0]
y2 = y[1]
y3 = y[2]

X_test = [np.concatenate([X[1], X[2]], axis=0), np.concatenate([X[0], X[2]], axis=0), np.concatenate([X[0], X[1]], axis=0)]
y_test = [np.concatenate((y[1], y[2])), np.concatenate((y[0], y[2])), np.concatenate((y[0], y[1]))]


subset_ranges = [np.arange(0, 4), np.arange(2, 6), np.arange(3, 8)]
subset_lengths = [subset_ranges[0].shape[0], subset_ranges[1].shape[0], subset_ranges[2].shape[0]]

In [7]:
X_test[0].shape

(2784, 8)

In [8]:
X_test[0].shape

(2784, 8)

In [9]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout, pout],[pout, pin, pout],[pout, pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs, seed=0)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10, 10]
#features_sizes = [8, 7, 6, 5]
pin = 0.5
pout = 0.1
seed = 0
alpha = 1e-2
lamda = 1e-1#1e-3#1e-1#1e-3
eta = 1 * 1e-2
d0 = min(subset_lengths)
no_users = sum(cluster_sizes)
batch_size = 50
epochs = 1
it = 500
G = generate_graph(cluster_sizes, pin, pout, seed)

# Set a random seed for reproducibility
seed = 17
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [10]:
d0

4

In [11]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [12]:
y1.shape

(1393,)

In [13]:
datapoints = {}
count = 0

X1, y1 = random_split(X1, y1, 10, 42)
X2, y2 = random_split(X2, y2, 10, 42)
X3, y3 = random_split(X3, y3, 10, 42)

X_train = [X1, X2, X3]
y_train = [y1, y2, y3]
input_sizes = [X1[0].shape[1], X2[0].shape[1], X3[0].shape[1]]

for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        
        test_features = X_train[i][j]#X_test[:, subset_ranges[i]]
        test_label = y_train[i][j]#y_test
        datapoints[count] = {
                'features': X_train[i][j],
                'degree': node_degree(count, G),
                'label': y_train[i][j],
                'neighbors': get_neighbors(count, G),
                'input_size': X_train[i][j].shape[1],
                'test_features':X_test[i][:, subset_ranges[i]],
                'test_labels': y_test[i]
            }
        count += 1

In [14]:
datapoints[21]['test_features'].shape

(2785, 5)

In [15]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [16]:

class MLP_Net(nn.Module):
    def __init__(self, input_size, num_classes, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(input_size, num_classes, bias=True)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        output = self.fc1(x)#F.softmax(self.fc1(x), dim=1)  # Applying softmax along the second dimension
        return output

In [17]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [18]:
model = MLP_Net(datapoints[21]["input_size"], 1, user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[21]["features"], datapoints[21]["label"]), batch_size=50, shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(1000):
    for j, (x, y) in zip(range(1), dataloader):
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)


        
        # Calculate accuracy
        #_, predicted = torch.max(yhat, 1)
        #_, predicted_true = torch.max(y, 1)
        #correct_predictions = (predicted == predicted_true).sum().item()
         

        loss = criterion(yhat.squeeze(), y)
        
        
        loss.backward()
        print(i, loss.detach())
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

0 tensor(96.4009)
1 tensor(120.1769)
2 tensor(111.0524)
3 tensor(91.6952)
4 tensor(94.1627)
5 tensor(94.1745)
6 tensor(80.5656)
7 tensor(69.3334)
8 tensor(70.9690)
9 tensor(71.9623)
10 tensor(56.2823)
11 tensor(52.8805)
12 tensor(55.0263)
13 tensor(45.1626)
14 tensor(45.2221)
15 tensor(45.4920)
16 tensor(44.4830)
17 tensor(41.0744)
18 tensor(47.4335)
19 tensor(36.9328)
20 tensor(38.0889)
21 tensor(34.5194)
22 tensor(27.8188)
23 tensor(34.5506)
24 tensor(35.8637)
25 tensor(41.8037)
26 tensor(25.5844)
27 tensor(30.0256)
28 tensor(32.2292)
29 tensor(37.1375)
30 tensor(31.9085)
31 tensor(31.6938)
32 tensor(37.3044)
33 tensor(30.9458)
34 tensor(17.9449)
35 tensor(29.8354)
36 tensor(27.1558)
37 tensor(33.5450)
38 tensor(32.5139)
39 tensor(30.4092)
40 tensor(22.7408)
41 tensor(25.3071)
42 tensor(26.2734)
43 tensor(35.3796)
44 tensor(24.6531)
45 tensor(23.9695)
46 tensor(34.5260)
47 tensor(32.8605)
48 tensor(32.5869)
49 tensor(18.6828)
50 tensor(22.8432)
51 tensor(24.4910)
52 tensor(18.6819)
5

421 tensor(9.5394)
422 tensor(9.0814)
423 tensor(8.5772)
424 tensor(11.6208)
425 tensor(9.0468)
426 tensor(7.1423)
427 tensor(14.6393)
428 tensor(11.9258)
429 tensor(9.6304)
430 tensor(7.2514)
431 tensor(12.4283)
432 tensor(6.9642)
433 tensor(15.0882)
434 tensor(6.6011)
435 tensor(9.1185)
436 tensor(11.8664)
437 tensor(12.8153)
438 tensor(10.7700)
439 tensor(7.2885)
440 tensor(13.2076)
441 tensor(7.9973)
442 tensor(12.6127)
443 tensor(16.0539)
444 tensor(9.9215)
445 tensor(6.0890)
446 tensor(6.2137)
447 tensor(14.3051)
448 tensor(8.5768)
449 tensor(4.5137)
450 tensor(13.5124)
451 tensor(9.8070)
452 tensor(9.1943)
453 tensor(8.4796)
454 tensor(10.6170)
455 tensor(8.8816)
456 tensor(14.6223)
457 tensor(12.3801)
458 tensor(7.7698)
459 tensor(9.2065)
460 tensor(9.3257)
461 tensor(8.6773)
462 tensor(8.8561)
463 tensor(7.3800)
464 tensor(8.1687)
465 tensor(6.6027)
466 tensor(12.0275)
467 tensor(7.5526)
468 tensor(8.9036)
469 tensor(11.6197)
470 tensor(11.1041)
471 tensor(16.3770)
472 tensor(

897 tensor(5.3784)
898 tensor(7.0077)
899 tensor(7.0254)
900 tensor(10.8680)
901 tensor(12.9191)
902 tensor(6.8479)
903 tensor(5.5857)
904 tensor(7.5197)
905 tensor(10.0674)
906 tensor(7.6830)
907 tensor(8.9906)
908 tensor(7.1564)
909 tensor(10.6470)
910 tensor(8.5121)
911 tensor(7.6536)
912 tensor(8.9909)
913 tensor(9.8533)
914 tensor(4.3170)
915 tensor(9.0209)
916 tensor(9.3485)
917 tensor(5.5439)
918 tensor(7.2766)
919 tensor(10.0440)
920 tensor(9.2125)
921 tensor(3.4506)
922 tensor(9.9528)
923 tensor(10.2199)
924 tensor(9.3591)
925 tensor(5.6228)
926 tensor(6.5693)
927 tensor(7.9162)
928 tensor(5.7938)
929 tensor(9.6396)
930 tensor(9.0934)
931 tensor(5.9705)
932 tensor(10.9917)
933 tensor(9.6978)
934 tensor(6.6489)
935 tensor(9.1834)
936 tensor(9.1887)
937 tensor(10.8827)
938 tensor(7.6818)
939 tensor(9.0877)
940 tensor(12.2672)
941 tensor(7.8016)
942 tensor(6.3757)
943 tensor(12.5605)
944 tensor(12.6414)
945 tensor(4.7607)
946 tensor(9.3541)
947 tensor(7.5439)
948 tensor(10.1912)


In [19]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=alpha)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            #model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output.squeeze(), labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[model.user_id][j] - projected_weights[j][model.user_id]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [20]:
# Preparing projection matrices
models = [MLP_Net(input_size=datapoints[i]['input_size'], num_classes=1, user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, models, first_run=True):
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        # Specify the dimensions of the rectangular matrix
                        row, column = d0, parameters_to_vector(models[i].parameters()).size()[0]

                        # Generate random values for the diagonal from a normal distribution
                        diag_values = torch.ones(min(row, column))

                        # Create a rectangular matrix with diagonal elements
                        mat = torch.diag(diag_values)

                        # If the matrix is larger than the diagonal vector, fill the remaining elements with zeros
                       
                        mat = torch.cat((mat, torch.zeros(row, column - row)), dim=1)

                        

                        # Append the matrix to the list
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[i].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[i][j], parameters_to_vector(models[i].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights, models)



In [21]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["test_features"], dataset["test_labels"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output.squeeze(), labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #_, predicted_true = torch.max(labels, 1)
        #correct += pred.eq(predicted_true.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    #correct /= len(test_loader.dataset)
    
    return test_loss#, correct

In [22]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(input_size=datapoints[i]['input_size'], num_classes=1, user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(input_size=datapoints[i]['input_size'], num_classes=1, user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

in_cluster_proj_norm = []
out_cluster_proj_norm = []
in_cluster_proj_diff_norm = []
out_cluster_proj_diff_norm = []
task_loss = {'0':[],
                '1':[],
                '2':[]}
task_rel_error = {'0':[],
            '1':[],
            '2':[]}

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    projected_weights = []
    update_ProjWeight(projection_list, projected_weights, models, first_run=False)
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            temp_mat = torch.outer(projected_weights[i][j] - projected_weights[j][i], weights)
            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * temp_mat)
    
    in_cluster_proj_norm_round = 0
    out_cluster_proj_norm_round = 0
    in_cluster_proj_diff_round = 0
    out_cluster_proj_diff_round = 0
    in_edges = 0
    out_edges = 0
    
    for i in range(no_users//3):
        for j in G.neighbors(i):
            if j < no_users//3:
                in_edges += 1
                in_cluster_proj_norm_round += torch.norm(projection_list[i][j]).detach().numpy()
                in_cluster_proj_diff_round += torch.norm(projected_weights[i][j] - projected_weights[j][i]).detach().numpy()
            else:
                out_edges += 1
                out_cluster_proj_norm_round += torch.norm(projection_list[i][j]).detach().numpy()
                out_cluster_proj_diff_round += torch.norm(projected_weights[i][j] - projected_weights[j][i]).detach().numpy()
    in_cluster_proj_norm.append(in_cluster_proj_norm_round / in_edges)
    out_cluster_proj_norm.append(out_cluster_proj_norm_round / out_edges)
    in_cluster_proj_diff_norm.append(in_cluster_proj_diff_round / in_edges)
    out_cluster_proj_diff_norm.append(out_cluster_proj_diff_round / out_edges)
            
                                         
                                              
    
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    per_task_loss = []
    per_task_rel_error = []
    
    for k in range(no_users):
      
        g_loss = testing(models[k], datapoints[k], 50, criterion)
        local_test_loss.append(g_loss)
        #user_rel_error += acc#rel_error(models[i])
        if (k + 1) % 10 == 0:
            task_loss[str(k // 10)].append(sum(per_task_loss) / 10)
            #task_rel_error[str(k // 10)].append(sum(per_task_rel_error) / 10)
            per_task_loss = []
            #per_task_rel_error = []
        per_task_loss.append(g_loss)
        #per_task_rel_error.append(acc)#rel_error(models[i]))
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    #total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f"% (test_loss[-1]))

  0%|          | 1/500 [00:01<08:55,  1.07s/it]

Training_loss 101.21219


  0%|          | 2/500 [00:02<08:55,  1.08s/it]

Training_loss 94.19116


  1%|          | 3/500 [00:03<08:22,  1.01s/it]

...

Training_loss 7.44968


100%|█████████▉| 499/500 [09:44<00:00,  1.15it/s]

Training_loss 7.44841


100%|██████████| 500/500 [09:45<00:00,  1.17s/it]

Training_loss 7.44543


In [23]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

in_cluster_proj_norm = np.array(in_cluster_proj_norm)
out_cluster_proj_norm = np.array(out_cluster_proj_norm)
in_cluster_proj_diff_norm = np.array(in_cluster_proj_diff_norm)
out_cluster_proj_diff_norm = np.array(out_cluster_proj_diff_norm)


In [24]:
'''
  0%|          | 1/2000 [00:12<6:59:49, 12.60s/it]
Training_loss 0.69317,   Accuracy 0.52177
  0%|          | 2/2000 [00:25<6:58:41, 12.57s/it]
Training_loss 0.69256,   Accuracy 0.52523
  0%|          | 3/2000 [00:37<6:55:30, 12.48s/it]
Training_loss 0.69282,   Accuracy 0.52400
  0%|          | 4/2000 [00:50<6:56:56, 12.53s/it]
Training_loss 0.69216,   Accuracy 0.52552
  0%|          | 5/2000 [01:05<7:35:34, 13.70s/it]
Training_loss 0.69178,   Accuracy 0.52765
  0%|          | 6/2000 [01:18<7:24:07, 13.36s/it]
Training_loss 0.69037,   Accuracy 0.55240
  0%|          | 7/2000 [01:32<7:32:02, 13.61s/it]
Training_loss 0.68986,   Accuracy 0.55653
  0%|          | 8/2000 [01:47<7:42:39, 13.94s/it]
Training_loss 0.68921,   Accuracy 0.56338
'''

'\n  0%|          | 1/2000 [00:12<6:59:49, 12.60s/it]\nTraining_loss 0.69317,   Accuracy 0.52177\n  0%|          | 2/2000 [00:25<6:58:41, 12.57s/it]\nTraining_loss 0.69256,   Accuracy 0.52523\n  0%|          | 3/2000 [00:37<6:55:30, 12.48s/it]\nTraining_loss 0.69282,   Accuracy 0.52400\n  0%|          | 4/2000 [00:50<6:56:56, 12.53s/it]\nTraining_loss 0.69216,   Accuracy 0.52552\n  0%|          | 5/2000 [01:05<7:35:34, 13.70s/it]\nTraining_loss 0.69178,   Accuracy 0.52765\n  0%|          | 6/2000 [01:18<7:24:07, 13.36s/it]\nTraining_loss 0.69037,   Accuracy 0.55240\n  0%|          | 7/2000 [01:32<7:32:02, 13.61s/it]\nTraining_loss 0.68986,   Accuracy 0.55653\n  0%|          | 8/2000 [01:47<7:42:39, 13.94s/it]\nTraining_loss 0.68921,   Accuracy 0.56338\n'

In [25]:
np.save( 'training_loss_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), test_loss)
np.save('relative_error_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), total_rel_error)
np.save( 'in_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), in_cluster_proj_norm)
np.save('out_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), out_cluster_proj_norm)
np.save( 'in_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), in_cluster_proj_diff_norm)
np.save('out_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), out_cluster_proj_diff_norm)

In [26]:
'training_loss_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'training_loss_sheave_fml0_1_pout0_1'

In [27]:
# task_loss dictionary
for key, value in task_loss.items():
    # Convert the list to a NumPy array
    array_loss = np.array(value)
    
    # Save the NumPy array using the specified format
    np.save('training_loss_sheave_fml_task' + key + '_' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), array_loss)

# task_rel_error dictionary
for key, value in task_rel_error.items():
    # Convert the list to a NumPy array
    array_rel_error = np.array(value)
    
    # Save the NumPy array using the specified format
    np.save('relative_error_sheave_fml_task' + key + '_' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), array_rel_error)